In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from yellowbrick.classifier import ConfusionMatrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler

from IPython.display import display, HTML
pd.options.display.max_columns=None

In [2]:
stock_data=pd.read_pickle("process_data/clean_data/sandp_ta.plk")

In [22]:
os.chdir(r"C:\Users\power\Dropbox\Cory")

In [3]:
quarterly_fin=pd.read_csv("SP500_quarterly_fin.zip")

date_vars=['datadate','fyearq','fqtr','fyr']

num_vars=['acchgq','acoq','actq','altoq','ancq','anoq',\
'aociderglq','aociotherq','aocisecglq','aol2q','aoq','apq','aqaq',\
'aqdq','aqepsq','aqpl1q','aqpq','arcedq','arceepsq','arceq','atq',\
'aul3q','billexceq','capr1q','capr2q','capsftq','capsq','capxy',\
'cdvcy','chechy','ceiexbillq','ceqq','cheq','chq','cibegniq','cicurrq',\
'ciderglq','cimiiq','ciotherq','ciy','cogsq','cshfd12','cshfdq','cshiq',\
'cshopq','cshoq','cshprq','cshtrq','cstkeq','cstkq','dcomq','dd1q','deracq',\
'deraltq','derhedglq','derlcq','derlltq','diladq','dlcq','dlttq','doq',\
'dpacreq','dpactq','dpq','dpretq','drcq','drltq','dteaq','dtedq','dteepsq',\
'dtepq','dvintfq','dvpq','dvpspq','epsfiq','epsfxy','esoptq','esubq','fcaq',\
'ffoq','finacoq','finchq','findlcq','findltq','finivstq','finltoq','finreccq',\
'finrecltq','finrevq','finxintq','finxoprq','gdwlamq','gdwliaq','gdwlidq',\
'gdwliepsq','gdwlipq','gdwlq','glaq','glceaq','glceepsq','glcepq','glivq',\
'glpq', 'hedgeglq','ibq','icaptq','intaccq','intanoq','intanq','invfgq','invtq',\
'itccy','ivaeqq','ivltq','ivstq','lctq','lltq','lnoq','lol2q','lul3q','ltq','mibnq',\
'mibq','mibtq','miiq','mkvaltq','msaq','ncoq','niitq','nimq','niq','nopiq','npatq',\
'npq','nrtxtepsq','nrtxtq','obkq','oepsxq','oiadpq','opepsq','piq','pllq',\
'pncepsq','pncpq','pstkq','rcaq','rcdq','recdq','recdq','rdipq','rectq','req',\
'revtq','rllq','rraq','saleq','setaq','spceq','spiq','tiiq','tieq','txdbaq','txdbclq',\
'txdbq','txwq','uaptq','uceqq','wcapq','wdaq','xaccq','xiq']

cat_vars=['gvkey','tic','cusip','idbflag','loc','naics','state']

quarterly_info=quarterly_fin[num_vars].describe().T

num_filtered=list(quarterly_info[quarterly_info['count']>13000].index)

quarterly_fin_filtered=quarterly_fin[date_vars+num_filtered+cat_vars]

quarterly_fin_filtered.loc[quarterly_fin_filtered['state'].isnull(),'state']=quarterly_fin_filtered['loc']

quarterly_fin_filtered.fillna(0, inplace=True)

quarterly_fin_filtered['year']=np.floor(quarterly_fin_filtered['datadate']/10000)
quarterly_fin_filtered['quarter']=np.floor((quarterly_fin_filtered['datadate']-quarterly_fin_filtered['year']*10000)/100)//3+quarterly_fin_filtered['year']*100

states=pd.get_dummies(quarterly_fin_filtered['state'])
states.columns=["state_"+str(i) for i in states.columns]

naics=pd.get_dummies(quarterly_fin_filtered['naics'])
naics.columns=["naics_"+str(i) for i in naics.columns]

idbflag=pd.get_dummies(quarterly_fin_filtered['idbflag'])
idbflag.columns=["idbflag_"+str(i) for i in idbflag.columns]

quarterly_fin_filtered=pd.concat([quarterly_fin_filtered, states, naics,idbflag], axis=1).drop(['idbflag','loc','naics','state'], axis=1)

quarterly_fin_filtered.to_pickle("quarterly_spy.pkl")

In [6]:
institutional=pd.read_csv("stock_institutional_ownership.csv", encoding='latin1')

institutionaltutional.head()

institutional_vars=['quarter','TIC','mktcap','io_usd', 'nbr_firms', 'io', \
                    'io_dom', 'io_for', 'io_for_us','io_for_nus', 'io_dom_indep', \
                    'io_dom_grey', 'io_for_indep',\
                    'io_for_grey', 'io_cat1', 'io_cat2', 'io_cat3', 'io_cat4', 'io_cat5',\
                    'io_cat6', 'io_cat7', 'io_indep', 'io_grey', 'io_common', 'io_civil',\
                    'io_for_common', 'io_for_civil', 'ibh_5pct', 'ibh_1pct', 'top5','herf']

institutional_filtered=institutional[institutional_vars]

institutional_filtered.rename(columns={'TIC':'tic'}, inplace=True)

institutional_filtered.drop_duplicates(['tic','quarter'], inplace=True)

fin_and_inst=quarterly_fin_filtered.merge(institutional_filtered, on=['tic','quarter'])

fin_and_inst.to_pickle("quarterly_spy_w_inst.pickle")

['.dropbox',
 'company_financial_info.zip',
 'desktop.ini',
 'execucomp',
 'Execucomp_Data_Definitions.pdf',
 'Fundamentals Quarterly Manual.html',
 'Fundamentals Quarterly Manual_files',
 'new.pkl',
 'quarterly_spy.pkl',
 'quarterly_spy_w_inst',
 'quarterly_spy_w_inst.pickle',
 'Security Daily Manual.html',
 'Security Daily Manual_files',
 'SP500_daily.zip',
 'SP500_quarterly_fin.zip',
 'stock_institutional_ownership.csv',
 'ticker_prices.zip']

In [20]:
quarterly_fin_filtered=pd.read_pickle("quarterly_spy_w_inst.pickle")

quarterly_fin_filtered['date']=pd.to_datetime(quarterly_fin_filtered['datadate'].astype(str),format='%Y%m%d')

quarterly_fin_filtered.set_index(['date'], inplace=True)

quarterly_fin_to_merge=quarterly_fin_filtered.groupby('tic').apply(lambda x: x.asfreq("D"))

merged=stock_data.drop('tic',axis=1).join(quarterly_fin_filled.drop('tic',axis=1), how='left')

merged=merged.groupby('tic', as_index=False).fillna(method='ffill')

merged=merged.reset_index()

merged_filled.dropna(inplace=True)

merged_filled.to_pickle("new.pkl")


KeyboardInterrupt



In [27]:
X = merged_filled.drop(['tic','date','close','high',\
                        'low','open','volume','nextwk_roi',\
                        'roi_class','datadate','fyearq','fqtr','fyr',], axis=1)

y = merged_filled.roi_class

X_train, X_test, y_train, y_test = train_test_split(Xp, y, test_size = 0.2)

sd = StandardScaler()
X_train = sd.fit_transform(X_train)
X_test = sd.fit_transform(X_test)

clf = LogisticRegression(n_jobs=-1, solver='lbfgs').fit(X_train, y_train)

C:\Users\power\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [28]:
print(clf.score(X_train, y_train))

print(clf.score(X_test, y_test))

0.7560421452112254
0.7303505442924199


In [29]:
X_non_PCA=X.loc[:, '30 period CCI_30': 'MFV_']
X_PCA=X.loc[:,'acchgq':]

poly = PolynomialFeatures(degree=2)
Xp = poly.fit_transform(X_non_PCA)

In [45]:
rbf_feature = RBFSampler(gamma=1, n_components=2000)

In [46]:
X_train_rbf=rbf_feature.fit_transform(X_train,)

In [49]:
svc=SVC(kernel='linear',C=1)

In [ ]:
svc.fit(X_train_rbf, y_train)

In [23]:
os.chdir(r"C:\Users\power\Dropbox\Cory")

In [24]:
full_data=pd.read_pickle("full_data.pkl")

In [25]:
full_data

,30 period CCI_30,21 period CCI_21,14 period CCI_14,MACD,SIGNAL,RSI_30,RSI_21,RSI_14,MFV_,roi_class,acchgq,acoq,actq,altoq,ancq,anoq,aociderglq,aociotherq,aocisecglq,aol2q,aoq,apq,aqpl1q,atq,aul3q,capsq,capxy,chechy,ceqq,cheq,chq,cibegniq,cicurrq,ciderglq,cimiiq,ciotherq,ciy,cogsq,cshfd12,cshfdq,cshiq,cshopq,cshoq,cshprq,cshtrq,cstkeq,cstkq,dcomq,diladq,dlcq,dlttq,doq,dpq,drcq,drltq,dvpq,dvpspq,epsfiq,epsfxy,esoptq,gdwlq,ibq,icaptq,intanoq,intanq,invtq,ivstq,lctq,lltq,lnoq,lol2q,lul3q,ltq,mibnq,mibq,mibtq,miiq,mkvaltq,msaq,niq,nopiq,oepsxq,oiadpq,opepsq,piq,pstkq,rdipq,rectq,req,revtq,saleq,spiq,txwq,wcapq,xiq,state_AL,state_AR,state_AZ,state_BMU,state_CA,state_CHE,state_CO,state_CT,state_DC,state_DE,state_FL,state_GA,state_GBR,state_IA,state_ID,state_IL,state_IN,state_IRL,state_KY,state_LA,state_MA,state_MD,state_ME,state_MI,state_MN,state_MO,state_NC,state_NE,state_NJ,state_NV,state_NY,state_OH,state_OK,state_OR,state_PA,state_RI,state_SC,state_TN,state_TX,state_USA,state_UT,state_VA,state_WA,state_WI,naics_42,naics_111,naics_315,naics_321,naics_423,naics_531,naics_2111,naics_2211,naics_3253,naics_3352,naics_3399,naics_4244,naics_22111,naics_31122,naics_31142,naics_31211,naics_31522,naics_33399,naics_33411,naics_33611,naics_54151,naics_54171,naics_211111,naics_212221,naics_212230,naics_212313,naics_212319,naics_213111,naics_213112,naics_221210,naics_221310,naics_236117,naics_236210,naics_237120,naics_237990,naics_238210,naics_311230,naics_311351,naics_311421,naics_311422,naics_311611,naics_311821,naics_311942,naics_312111,naics_312120,naics_312140,naics_312230,naics_314110,naics_315220,naics_316210,naics_316992,naics_322121,naics_322130,naics_322211,naics_322220,naics_324110,naics_325120,naics_325180,naics_325199,naics_325211,naics_325220,naics_325311,naics_325320,naics_325412,naics_325413,naics_325414,naics_325510,naics_325611,naics_325612,naics_325620,naics_326113,naics_326211,naics_331110,naics_332216,naics_332431,naics_332510,naics_332912,naics_332913,naics_333111,naics_333120,naics_333132,naics_333242,naics_333314,naics_333316,naics_333415,naics_333618,naics_333914,naics_333991,naics_334111,naics_334112,naics_334118,naics_334210,naics_334220,naics_334413,naics_334417,naics_334419,naics_334510,naics_334511,naics_334513,naics_334516,naics_334517,naics_334519,naics_335122,naics_335220,naics_335314,naics_336120,naics_336310,naics_336320,naics_336411,naics_336412,naics_336413,naics_336414,naics_336611,naics_336991,naics_337110,naics_337910,naics_339112,naics_339113,naics_339114,naics_339115,naics_339930,naics_423120,naics_423140,naics_423450,naics_424210,naics_441120,naics_441310,naics_443142,naics_444110,naics_445110,naics_446110,naics_446120,naics_448140,naics_448190,naics_448210,naics_448310,naics_452210,naics_452311,naics_452319,naics_453998,naics_454110,naics_481111,naics_482111,naics_483112,naics_484121,naics_486210,naics_488510,naics_492110,naics_511110,naics_511210,naics_512110,naics_515120,naics_515210,naics_517311,naics_517312,naics_518210,naics_519130,naics_522110,naics_522210,naics_522298,naics_522320,naics_523110,naics_523120,naics_523210,naics_523920,naics_523930,naics_523999,naics_524113,naics_524114,naics_524126,naics_524130,naics_524210,naics_524298,naics_531110,naics_531120,naics_531130,naics_531190,naics_532282,naics_532412,naics_541213,naics_541214,naics_541512,naics_541611,naics_541810,naics_541910,naics_561320,naics_561450,naics_561510,naics_562111,naics_562211,naics_621491,naics_621492,naics_621493,naics_621511,naics_622110,naics_721110,naics_721120,naics_722511,naics_722513,naics_812331,naics_999977,idbflag_B,idbflag_D,mktcap,io_usd,nbr_firms,io,io_dom,io_for,io_for_us,io_for_nus,io_dom_indep,io_dom_grey,io_for_indep,io_for_grey,io_cat1,io_cat2,io_cat3,io_cat4,io_cat5,io_cat6,io_cat7,io_indep,io_grey,io_common,io_civil,io_for_common,io_for_civil,ibh_5pct,ibh_1pct,top5,herf,30 period CCI_30_sandp,21 period CCI_21_sandp,14 period CCI_14_sandp,MACD_sandp,SIGNAL_sandp,RSI_30_sandp,RSI_21_sandp,RSI_14_sand